Id : ked
Pwd : ked0911

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import json
import time
from datetime import date,timedelta
import os
from collections import Counter
from difflib import SequenceMatcher
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore') #경고문구 무시


In [2]:
#하루 수집
def get_news_per_day(company, daily_date):
    url = 'https://api.deepsearch.com/v1/compute?input=DocumentSearch(["news"],[""],"title:({})",count=9999999999999, date_from={},date_to={})'.format(company, str(daily_date), str(daily_date) )
    headers = {'Authorization': 'Basic {YTM0N2JjN2UwMzhhNGEwYWJjNTg4M2Y0NWVhZDhiODg6MjJkZjM3ZjRkMDAyNzhlZGFhMjE2NzU3YmEzYTA3MTdjMTE2ZjI3NWM3MDUwNDFmZjNlZmUwYWM0ODNlYTk4ZA==}'}

    response = requests.get(url, headers=headers, verify=False)
    elements = BeautifulSoup(response.content, 'html.parser', from_encoding='UTF-8')
    elements_json = json.loads(elements.text)

    data = []

    for d in elements_json['data']['pods'][1]['content']['data']['docs']:
        try:
            attachments = d.get('attachments')
            if attachments == []:
                attachments = ''
            author = d.get('author')
            category = d.get('category')
            date = d.get('created_at').split('T')[0]
            if d.get('industry'):
                industry_label = d.get('industry').get('label')
                industry_name = d.get('industry').get('name')
                industry_score = d.get('industry').get('score')
            else:
                industry_label = ''
                industry_name = ''
                industry_score = ''
            if d.get('polarity'):
                polarity_label = d.get('polarity').get('label')
                polarity_name = d.get('polarity').get('name')
                polarity_score = d.get('polarity').get('score')
            else:
                polarity_label = ''
                polarity_name = ''
                polarity_score = ''
            publisher = d.get('publisher')
            section = d.get('section')
            title = d.get('title')
            content_url = d.get('content_url')
        except:
            pass

        data.append({
            "company" : company,
            "attachments" : attachments,
            'author' : author,
            'category' : category,
            'date' : date,
            "industry_label" : industry_label,
            'industry_name' : industry_name,
            'industry_score' : industry_score,
            'polarity_label' : polarity_label,        
            'polarity_name' : polarity_name,        
            'polarity_score' : polarity_score,
            'publisher' : publisher,
            'section' : section,
            'title' : title,
            'content_url' : content_url
        })
    df = pd.DataFrame(data)
    
    return df

#기간 수집
def get_news_per_days(company, from_date, to_date):
    url = 'https://api.deepsearch.com/v1/compute?input=DocumentSearch(["news"],[""],"title:({})",count=9999999999999, date_from={},date_to={})'.format(company, str(from_date), str(to_date) )
    headers = {'Authorization': 'Basic {YTM0N2JjN2UwMzhhNGEwYWJjNTg4M2Y0NWVhZDhiODg6MjJkZjM3ZjRkMDAyNzhlZGFhMjE2NzU3YmEzYTA3MTdjMTE2ZjI3NWM3MDUwNDFmZjNlZmUwYWM0ODNlYTk4ZA==}'}

    response = requests.get(url, headers=headers, verify=False)
    elements = BeautifulSoup(response.content, 'html.parser', from_encoding='UTF-8')
    elements_json = json.loads(elements.text)

    data = []

    for d in elements_json['data']['pods'][1]['content']['data']['docs']:
        try:
            attachments = d.get('attachments')
            if attachments == []:
                attachments = ''
            author = d.get('author')
            category = d.get('category')
            date = d.get('created_at').split('T')[0]
            if d.get('industry'):
                industry_label = d.get('industry').get('label')
                industry_name = d.get('industry').get('name')
                industry_score = d.get('industry').get('score')
            else:
                industry_label = ''
                industry_name = ''
                industry_score = ''
            if d.get('polarity'):
                polarity_label = d.get('polarity').get('label')
                polarity_name = d.get('polarity').get('name')
                polarity_score = d.get('polarity').get('score')
            else:
                polarity_label = ''
                polarity_name = ''
                polarity_score = ''
            publisher = d.get('publisher')
            section = d.get('section')
            title = d.get('title')
            content_url = d.get('content_url')
        except:
            pass

        data.append({
            "company" : company,
            "attachments" : attachments,
            'author' : author,
            'category' : category,
            'date' : date,
            "industry_label" : industry_label,
            'industry_name' : industry_name,
            'industry_score' : industry_score,
            'polarity_label' : polarity_label,        
            'polarity_name' : polarity_name,        
            'polarity_score' : polarity_score,
            'publisher' : publisher,
            'section' : section,
            'title' : title,
            'content_url' : content_url
        })
    df = pd.DataFrame(data)
    
    return df

In [3]:
#유사도 측정
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

#중복 기사 제거
def del_dulicated_news(ked_ordered_news):
    sensitivity = 0.6
    compared = pd.DataFrame(columns=["기준일자","구분","제목","출처","URL"])
    print("-------------------------------민감도"+str(int(sensitivity*100))+"% 중복인식제목들-------------------------------")
    while not ked_ordered_news.empty:
        comparing = ked_ordered_news.iloc[0]
        compared.loc[len(compared)]= comparing
        ked_ordered_news=ked_ordered_news[1:]
        ked_ordered_news=ked_ordered_news.reset_index(drop=True)
        dup_list = []
        for idx,i in enumerate(ked_ordered_news.values):
            if similar(comparing[2],i[2]) > sensitivity: #유사도가 0.6보다 클 때
                    print((comparing[2],i[2]))
                    dup_list.append(idx)
        if len(dup_list)!=0:
            ked_ordered_news = ked_ordered_news.drop(dup_list).reset_index(drop=True)
    return compared

#하루 뉴스
def ked_news(day):
    keyword = pd.read_excel("키워드.xlsx") #키워드 파일 읽어오기
    order_list = pd.read_excel('뉴스우선순위.xlsx') #뉴스 우선순위 파일 읽어오기
    dfs = list()
    for i in keyword.values:
        temp = get_news_per_day(i[1],day)
        if len(temp)!=0:
            temp['keyword'] = i[0]
            dfs.append(temp)
    if len(dfs)!=0:
        news = pd.concat(dfs)
        news['publisher'] = '[' + news['publisher'].astype(str) + ']' #출처 중괄호 추가
        df_file = news[['date','keyword','title','publisher','content_url']]
        df_file.columns = ["기준일자","구분","제목","출처","URL"]
    else:
        print("There is no news")
    for col_i in df_file:
        df_file[col_i] = df_file[col_i].apply(lambda x: x.encode('euc_kr', 'ignore').decode('euc_kr'))
    sorter = list(order_list.출처)
    df_file.출처 = df_file.출처.astype("category")
    df_file.출처.cat.set_categories(sorter, inplace=True)
    ordered_news = df_file.sort_values(["출처"]).reset_index(drop=True)
    ordered_news = del_dulicated_news(ordered_news)
    ordered_news.to_excel("경영환경동향"+day+".xlsx",encoding='euc-kr',index=False)
    return ordered_news


#기간 뉴스
def ked_news_days(from_day,to_day):
    keyword = pd.read_excel("키워드.xlsx") #키워드 파일 읽어오기
    order_list = pd.read_excel('뉴스우선순위.xlsx') #뉴스 우선순위 파일 읽어오기
    dfs = list()
    for i in keyword.values:
        temp = get_news_per_days(i[1],from_day,to_day)
        if len(temp)!=0 :
            temp['keyword'] = i[0]
            dfs.append(temp)
    if len(dfs)!=0 :
        news = pd.concat(dfs)
        news['publisher'] = '[' + news['publisher'].astype(str) + ']' #출처 중괄호 추가
        df_file = news[['date','keyword','title','publisher','content_url']]
        df_file.columns = ["기준일자","구분","제목","출처","URL"]
    else:
        print("There is no news")
    for col_i in df_file:
        df_file[col_i] = df_file[col_i].apply(lambda x: x.encode('euc_kr', 'ignore').decode('euc_kr'))
    sorter = list(order_list.출처)
    df_file.출처 = df_file.출처.astype("category")
    df_file.출처.cat.set_categories(sorter, inplace=True)
    ordered_news = df_file.sort_values(["출처"]).reset_index(drop=True)
    ordered_news = del_dulicated_news(ordered_news)
    ordered_news.to_excel("경영환경동향"+from_day+"~"+to_day+".xlsx",encoding='euc-kr',index=False)
    return ordered_news



In [4]:
#----------------------------------------------오늘 날짜 선택-----------------------------------------------------------------
day = date.today().isoformat() #: 오늘날짜

#----------------------------------------------함수 선택-----------------------------------------------------------------
ked_news(day) # 해당 일 수집
# ked_news_days('2021-04-27','2021-04-27') # 해당 기간 수집

-------------------------------민감도60% 중복인식제목들-------------------------------
("해성디에스, ESG 평가 5년 연속 'A' 등급 달성", 'CJ대한통운, ESG 평가 2년 연속 A등급')
("현대차증권, 한국기업지배구조원 ESG평가 2년 연속 'A'", "CJ대한통운, 韓 기업지배구조원 ESG 평가 2년 연속 'A' 우수등급")
("현대차증권, 한국기업지배구조원 ESG평가 2년 연속 'A'", 'CJ대한통운, 기업지배구조원 ESG 평가 2년 연속 ‘A’ 등급')
("현대차증권, 한국기업지배구조원 ESG평가 2년 연속 'A'", 'CJ대한통운, 기업지배구조원 ESG 평가 2년 연속 A등급')
('GS리테일, 2021년 ESG 종합등급 \'A\' 획득.. "지속 가능 기업으로 성장"', "GS리테일, ESG 종합등급 'A' 획득…지속가능경영 결실")
('GS리테일, 2021년 ESG 종합등급 \'A\' 획득.. "지속 가능 기업으로 성장"', 'GS리테일, ESG 종합 등급 ‘A’ 획득…“지속 가능한 기업으로”')


기준일자     구분                                              제목  \
0   2021-10-27  평가서비스                         aT-서울교육청, ESG실천 업무협약 체결   
1   2021-10-27   주주사                             대한상의 ‘제37차 금융위원회’ 개최   
2   2021-10-27  평가서비스                   한국리츠 20년 맞아... 리츠협회 ESG 경영 선포   
3   2021-10-27  평가서비스                   해성디에스, ESG 평가 5년 연속 'A' 등급 달성   
4   2021-10-27  평가서비스                현대차증권, 한국기업지배구조원 ESG평가 2년 연속 'A'   
5   2021-10-27   주주사                         [기자24시] 기재부, 예타 수행 자격 있나   
6   2021-10-27  평가서비스                   쌍용C&E, 신한은행과 ESG 상생 금융지원 업무협약   
7   2021-10-27  평가서비스             사모로만 발행하던 이랜드월드, 1000억 ESG 공모채 '도전'   
8   2021-10-27  평가서비스                     야놀자의 ESG, 대전 친환경 사무실 리뉴얼 오픈   
9   2021-10-27  평가서비스  GS리테일, 2021년 ESG 종합등급 'A' 획득.. "지속 가능 기업으로 성장"   
10  2021-10-27  평가서비스                LG디스플레이, 1차 ESG위원회 개최…이창양 위원장 선임   
11  2021-10-27  평가서비스              남양주 ESG행정 한 축 '두번째옷장', 농·축협들도 힘 보태   
12  2021-10-27   주주사                         “금융당국 대출 규제…증권업·증시엔 부정적”   
13  2021-10-27   주주사                    '시즌 첫 승' 먼저 웃은 도로공사…기업은행은 3연패   
14  2021-10-27   주주사                    기업은행, 작년 순익 '너끈'…가볍게 2조 클럽 예약   
15  2021-10-27  평가서비스                  LG화학, 협력사 ESG 지원 ‘1000억 펀드’ 조성   
16  2021-10-27  평가서비스                      롯데손보, ESG 평가에서 통합 ‘A’등급 획득   
17  2021-10-27  평가서비스              이명재 오자 달라졌다… 롯데손해보험, ESG평가 A등급 받았다   
18  2021-10-27  평가서비스                          LGD도 ESG…제1차 ESG위원회 개최   
19  2021-10-27  평가서비스            디지털로 ESG경영 지원, IT서비스업계 ESG 플랫폼 출시 속도   
20  2021-10-27  평가서비스                  ‘ESG 투자하면 PBR 상승’…일본 제약사 실증 분석   
21  2021-10-27   주주사                   키움증권, 지난 사업보고서 회계 오류로 금감원 '경고'   
22  2021-10-27   주주사                    ‘미녀’ 라셈 고군분투에도…IBK기업은행 3연패 수렁   

          출처                                                URL  
0     [세계일보]  http://www.segye.com/content/html/2021/10/26/2...  
1     [세계일보]  http://www.segye.com/content/html/2021/10/26/2...  
2     [세계일보]  http://www.segye.com/content/html/2021/10/26/2...  
3      [뉴시스]  http://www.newsis.com/view/?id=NISX20211027_00...  
4      [뉴시스]  http://www.newsis.com/view/?id=NISX20211027_00...  
5     [매일경제]  http://news.mk.co.kr/newsRead.php?no=1014954&y...  
6     [매일경제]  http://news.mk.co.kr/newsRead.php?no=1015282&y...  
7    [헤럴드경제]  http://news.heraldcorp.com/view.php?ud=2021102...  
8   [파이낸셜뉴스]      http://www.fnnews.com/news/202110270829340470  
9   [파이낸셜뉴스]      http://www.fnnews.com/news/202110270905329271  
10   [아시아경제]  https://view.asiae.co.kr/article/2021102708140...  
11    [이데일리]  http://www.edaily.co.kr/news/newspath.asp?news...  
12    [이데일리]  http://www.edaily.co.kr/news/newspath.asp?news...  
13  [연합뉴스TV]  http://www.yonhapnewstv.co.kr/MYH2021102700240...  
14  [비즈니스워치]  http://news.bizwatch.co.kr/article/finance/202...  
15    [서울신문]  https://www.seoul.co.kr/news/newsView.php?id=2...  
16    [조세일보]  http://www.joseilbo.com/news/news_read.php?uid...  
17     [머니S]  http://moneys.mt.co.kr/news/mwView.php?no=2021...  
18  [디지털데일리]  https://www.ddaily.co.kr/news/article.html?no=...  
19  [디지털데일리]  https://www.ddaily.co.kr/news/article.html?no=...  
20  [한경비즈니스]  https://magazine.hankyung.com/business/article...  
21  [아이뉴스24]                http://www.inews24.com/view/1415495  
22       NaN           http://mksports.co.kr/view/2021/1014986/

In [4]:
#----------------------------------------------어제 날짜 선택-----------------------------------------------------------------
day = (date.today() - timedelta(days=1)).isoformat() #: 어제날짜

#----------------------------------------------함수 선택-----------------------------------------------------------------
ked_news(day) # 해당 일 수집
# ked_news_days('2021-04-27','2021-04-27') # 해당 기간 수집

-------------------------------민감도60% 중복인식제목들-------------------------------
('발표하는 이승원 기재부 국고국 심의관', '발표하는 이승원 기재부 국고국 심의관')
('발표하는 이승원 기재부 국고국 심의관', '발표하는 이승원 기재부 국고국 심의관')
('홍남기 부총리, 기재부 확대간부회의 주재', '홍남기 부총리, 기재부 확대간부회의 주재')
('홍남기 부총리, 기재부 확대간부회의 주재', "홍남기 부총리 '기획재정부 확대간부회의'")
('홍남기 부총리, 기재부 확대간부회의 주재', '홍남기 부총리, 기획재정부 확대간부회의')
('홍남기 부총리, 기재부 확대간부회의 주재', '기재부 확대간부회의')
('기획재정부 확대간부회의', '기획재정부 확대간부회의 주재하는 홍남기 부총리')
('기획재정부 확대간부회의', '기획재정부 확대간부회의 주재하는 홍남기 부총리')
('기획재정부 확대간부회의', '기획재정부 확대간부회의')
('기획재정부 확대간부회의', '기획재정부 확대간부회의')
('대한상의 금융위원회 37차 회의', '대한상의 금융위원회 회의')
('대한상의 금융위원회 37차 회의', '대한상의 금융위원회 37차 회의')
('대한상의 금융위원회 37차 회의', '대한상의 금융위원회 회의')
('대한상의 금융위원회 37차 회의', '대한상공회의소 금융위원회 제37차 회의')
('대한상의 금융위원회 37차 회의', '제37차 대한상의 금융위원회')
('대한상의 금융위원회 37차 회의', "'대한상의 금융위원회’")
('KCC글라스, 지속가능경영 성과 담긴 ESG보고서 첫 발간', 'KCC글라스, 첫 번째 ESG보고서 발간')
('KCC글라스, 지속가능경영 성과 담긴 ESG보고서 첫 발간', 'KCC글라스, 첫 번째 ESG보고서 발간')
('KCC글라스, 지속가능경영 성과 담긴 ESG보고서 첫 발간', 'KCC글라스, 첫번째 ESG보고서 발간')
('KCC글라스, 지속가능경영 성과 담긴 ESG보고서 첫 발

기준일자     구분                                           제목        출처  \
0    2021-10-26   주주사         전세대출, 손도 못 댄 금융위…저소득·청년만 불리 '부작용' 우려도    [한국일보]   
1    2021-10-26   주주사                          발표하는 이승원 기재부 국고국 심의관    [연합뉴스]   
2    2021-10-26   주주사                        홍남기 부총리, 기재부 확대간부회의 주재    [연합뉴스]   
3    2021-10-26   주주사                                  기획재정부 확대간부회의    [연합뉴스]   
4    2021-10-26   주주사                             대한상의 금융위원회 37차 회의    [연합뉴스]   
..          ...    ...                                          ...       ...   
108  2021-10-26  평가서비스       아워홈, ESG 경영 가속화…"테이크아웃 메뉴에 친환경 패키징 도입"  [아이뉴스24]   
109  2021-10-26   주주사   '시즌 첫 승리' 도로공사, IBK기업은행 3:1로 제쳐...켈시 28점 활약       NaN   
110  2021-10-26   주주사           켈시 28점 도로공사 첫승…김희진·김수지·표승주 기업은행 3연패       NaN   
111  2021-10-26   주주사             [MD포토] 시즌 첫 승 한국도로공사와 3연패 IBK기업은행       NaN   
112  2021-10-26   주주사             '연패 탈출과 시즌 첫 승' IBK기업은행 vs 한국도로공사       NaN   

                                                   URL  
0    https://hankookilbo.com/News/Read/A20211026141...  
1         http://yna.kr/PYH20211026229300013?did=1196m  
2         http://yna.kr/PYH20211026189100013?did=1196m  
3         http://yna.kr/PYH20211026188900013?did=1196m  
4         http://yna.kr/PYH20211026038900013?did=1196m  
..                                                 ...  
108                http://www.inews24.com/view/1415564  
109  http://www.thesportstimes.co.kr/news/articleVi...  
110  https://www.mhnse.com/news/articleView.html?id...  
111  http://www.mydaily.co.kr/new_yk/html/read.php?...  
112  http://www.thesportstimes.co.kr/news/articleVi...  

[113 rows x 5 columns]